In [1]:
import os
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "0" #use GPU:0 only
gpus = tf.config.experimental.list_physical_devices(device_type='GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu,enable=True) 

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
#print GPU Devices
[print(x) for x in local_device_protos if x.device_type == 'GPU']
print('Tensorflow Version:',tf.__version__)

In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplo

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10561793600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11678055157591951821
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5"

Tensorflow Version: 2.4.1


# Preprocessing

In [2]:
# load_data
data_la = np.load('/data1/qzhang/small_dataset_pnas/original/original_dataset.npz')['la']
labels = np.load('/data1/qzhang/small_dataset_pnas/original/original_dataset.npz')['labels']

# oever view
print('Original la shape:',data_la.shape)
print('labels:',np.unique(labels))

Original la shape: (2700, 3000)
labels: ['a-helix' 'b-sheet' 'other-SS']


In [3]:
# preprocess labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_cat = encoder.fit_transform(labels)
print(encoder.classes_)
print(np.unique(labels_cat))

['a-helix' 'b-sheet' 'other-SS']
[0 1 2]


In [4]:
# preprocess la data
# preprocess

from sklearn.preprocessing import minmax_scale
data_la =  minmax_scale(data_la,axis=1)
print('Max per sample:',data_la[0].max())
print('Min per sample:',data_la[0].min())


# split dataset
from sklearn.model_selection import train_test_split
train_data,test_data,train_label,test_label = train_test_split(data_la.reshape(-1,3000,1),labels_cat,
                                                               stratify = labels_cat,
                                                               test_size=0.2,
                                                               random_state=42)

train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,stratify=train_label,
                                               test_size = 0.125,random_state=42)


print('Train data:',train_x.shape)
print('Validation data:',val_x.shape)
print('Test data:',test_data.shape)


Max per sample: 1.0000000000000002
Min per sample: 0.0
Train data: (1890, 3000, 1)
Validation data: (270, 3000, 1)
Test data: (540, 3000, 1)


# Build a 1D CNN Demo

In [5]:
def build_model():

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(64,10,activation='relu',input_shape=(3000,1),padding='same'),
        tf.keras.layers.MaxPooling1D(10),
        tf.keras.layers.Conv1D(64,10,activation='relu',padding='same'),
        tf.keras.layers.MaxPooling1D(10),
        tf.keras.layers.Dropout(0.25),
        # tf.keras.layers.LSTM(32,return_sequences=True),
        # tf.keras.layers.LSTM(32),
        # tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(3,activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',
                    metrics=['acc'])
    return model

In [6]:
model = build_model()
history = model.fit(train_x,
                   train_y, epochs=50, batch_size=64,
                   validation_data=(val_x,val_y))

Epoch 1/50
30/30 [==============================] - 4s 28ms/step - loss: 1.0926 - acc: 0.3610 - val_loss: 1.0708 - val_acc: 0.4444
Epoch 2/50
30/30 [==============================] - 0s 9ms/step - loss: 1.0709 - acc: 0.4333 - val_loss: 1.0001 - val_acc: 0.6444
Epoch 3/50
30/30 [==============================] - 0s 10ms/step - loss: 0.9976 - acc: 0.5496 - val_loss: 0.8856 - val_acc: 0.6407
Epoch 4/50
30/30 [==============================] - 0s 10ms/step - loss: 0.9223 - acc: 0.5960 - val_loss: 0.7948 - val_acc: 0.6852
Epoch 5/50
30/30 [==============================] - 0s 10ms/step - loss: 0.8321 - acc: 0.6464 - val_loss: 0.7250 - val_acc: 0.7111
Epoch 6/50
30/30 [==============================] - 0s 9ms/step - loss: 0.7890 - acc: 0.6610 - val_loss: 0.6791 - val_acc: 0.7259
Epoch 7/50
30/30 [==============================] - 0s 10ms/step - loss: 0.7439 - acc: 0.6921 - val_loss: 0.6483 - val_acc: 0.7407
Epoch 8/50
30/30 [==============================] - 0s 10ms/step - loss: 0.7371 - acc

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 3000, 64)          704       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 300, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 64)           41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 30, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

### Test test data

In [8]:
model.evaluate(test_data,test_label)

17/17 [==============================] - 0s 5ms/step - loss: 0.4705 - acc: 0.8056


[0.4704816937446594, 0.8055555820465088]

### save demo model

In [9]:
model.save('./1DCNNDemoModel.h5')